In [40]:
import datetime, time, math, os, json, shutil, asyncio, networkx as nx
from utils.gitutils import create_pull_request, clone_repo, create_branch, wipe_repo, prepare_repo
from utils.agent import Agent, GenerationConfig, Interaction, Team
from utils.stringutils import arr_from_sep_string, extract_markdown_blocks, markdown_to_dict
from utils.filetreeutils import FileTree, write_file_tree
from utils.listutils import flatten
from utils.frameworkutils import DartAnalyzer
from dotenv import load_dotenv
from utils.graphutils import loose_level_order, collapsed_level_order
from utils.frameworkutils import Framework

In [41]:
load_dotenv()  # Load environment variables from .env

GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')
API_KEY = os.getenv('API_KEY')

In [42]:
repo_url = 'https://github.com/6165-MSET-CuttleFish/TeamTrack'
source = Framework.FLUTTER
feature = f"""# Name: Organizations
# Description: Organizations are a way to group users together. They can be used to automatically share events with the members of the organization whenever they are uploaded to the cloud. \
Organizations consist of a name, team number, and a backend-generated unique id. They can also have a profile picture, description, and location. \
Members can also opt to be publicly listed as a member of the organization or not. \
"""

In [43]:
# Clone the repo and make a new branch
repo = clone_repo(repo_url)
base_branch = repo.active_branch.name
created_branch = f"modification-{base_branch}"
local_repo_path = str(repo.working_dir)
working_dir_path = f'{local_repo_path}\\{source.get_working_dir()}'

In [44]:
source_swe_summarizer = Agent(
    model_name="gemini-1.5-flash-001",
    api_key=API_KEY,
    name="source_swe_analyzer",
    generation_config=GenerationConfig(temperature=0.7),
    system_prompt=f'''You are a {source} software engineer. Your job is to summarize the functionality of provided code files. Structure your response as follows:

# filename.ext:

$7 sentence summary of the functionality/contents of the file$
'''
)

pm = Agent(
    model_name="gemini-1.5-pro-001",
    api_key=API_KEY,
    name="pm",
    generation_config=GenerationConfig(temperature=0.3),
    system_prompt=f"""You are a high-level technical project manager tasked with the project of adding functionality to a {source} repository."""
)

source_swe_coder = Agent(
    model_name="gemini-1.5-flash-001",
    api_key=API_KEY,
    name="source_swe_coder",
    generation_config=GenerationConfig(temperature=0.7),
    system_prompt=f'''You are a {source} software engineer. Your job is to write code to implement a specific feature. Structure your responses as follows:
# Code
```
$code$
```
# Example Usage
```
$example usage$
```
'''
)

team = Team(source_swe_summarizer, pm, source_swe_coder)

In [45]:
analyzer = source.get_analyzer(working_dir_path)

source_dependency_graph = analyzer.buildDependencyGraph()

In [46]:
# loose level order
eval_order = loose_level_order(source_dependency_graph)[::-1]
for level in eval_order:
    print(level)
print(f"Number of levels: {len(eval_order)}")

['models\\AppModel.dart', 'models\\StatConfig.dart']
['models\\GameModel.dart']
['models\\ScoreModel.dart']
['components\\misc\\PlatformGraphics.dart', 'functions\\Extensions.dart']
['components\\users\\PFP.dart', 'components\\statistics\\BarGraph.dart']
['components\\scores\\Incrementor.dart', 'components\\users\\UsersRow.dart', 'providers\\Auth.dart', 'functions\\Statistics.dart', 'components\\scores\\ScoreSummary.dart']
['views\\home\\match\\MatchView.dart']
['components\\statistics\\PercentChange.dart', 'functions\\Functions.dart']
['components\\scores\\ScoreRangeSummary.dart', 'components\\scores\\ScoreTimeline.dart', 'models\\Change.dart']
['components\\misc\\EmptyList.dart', 'views\\home\\match\\ExampleMatchRow.dart', 'components\\statistics\\CheckList.dart', 'components\\scores\\ScoringElementStats.dart', 'views\\home\\match\\MatchConfig.dart', 'views\\home\\change\\ChangeRow.dart', 'components\\misc\\CardView.dart', 'views\\home\\match\\MatchRow.dart', 'views\\home\\change\\Ch

In [47]:
(source_file_tree := FileTree.from_dir(working_dir_path))

├── api\
│   ├── README.MD
├── components\
│   ├── misc\
│   │   ├── CardView.dart
│   │   ├── Collapsible.dart
│   │   ├── EmptyList.dart
│   │   ├── InfoPills.dart
│   │   ├── PlatformGraphics.dart
│   ├── scores\
│   │   ├── Incrementor.dart
│   │   ├── ScoreCard.dart
│   │   ├── ScoreRangeSummary.dart
│   │   ├── ScoreSummary.dart
│   │   ├── ScoreTimeline.dart
│   │   ├── ScoringElementStats.dart
│   ├── statistics\
│   │   ├── BarGraph.dart
│   │   ├── CheckList.dart
│   │   ├── PercentChange.dart
│   ├── users\
│   │   ├── PFP.dart
│   │   ├── UsersRow.dart
│   ├── AutonomousDrawingTool.dart
├── functions\
│   ├── APIMethods.dart
│   ├── Extensions.dart
│   ├── Functions.dart
│   ├── GPTModel.dart
│   ├── ResponseModel.dart
│   ├── Statistics.dart
├── models\
│   ├── AppModel.dart
│   ├── Change.dart
│   ├── GameModel.dart
│   ├── GPTModel.dart
│   ├── ScoreModel.dart
│   ├── StatConfig.dart
├── providers\
│   ├── Auth.dart
│   ├── PushNotifications.dart
│   ├── Theme.dart
│   ├

In [48]:
# Summarize the original repo
from tqdm import tqdm

async def summarize_group(group):
    async def summarize(node):
        name = source_file_tree.nodes[node]['name']
        content = source_file_tree.nodes[node]['content']
        message = f"{name}\n```\n{content}\n```"
        return await team.async_chat_with_agent(
            agent_name='source_swe_analyzer',
            message=message,
            context_keys=[f"summary_{neighbor}" for neighbor in source_dependency_graph[node]],
            save_keys=[f"summary_{node}", "all"],
            prompt_title=f"Summary of {node}"
            )
    tasks = [summarize(node) for node in group]
    responses = await asyncio.gather(*tasks)
    for i, response in enumerate(responses):
        source_file_tree.nodes[group[i]]["summary"] = response

for level in tqdm(eval_order):
    await summarize_group(level)
    time.sleep(0.5)

100%|██████████| 19/19 [01:22<00:00,  4.37s/it]


In [57]:
prompt = f"""This is the file tree for the original {source} repo:
```
{source.get_working_dir()}\\
{source_file_tree}
```
Given the prior context, summarize the functionality of the entire repository succinctly.
"""
print(team.chat_with_agent('source_swe_analyzer', prompt, context_keys=["all"], save_keys=["all"], prompt_title=f"""Repository Summary.
File Tree provided:
```
{source.get_working_dir()}\\
{source_file_tree}
```
"""))

This repository houses a Flutter application called "TeamTrack," designed to streamline the management and analysis of data for FIRST Tech Challenge (FTC) robotics competitions. 

The app provides features for:

- **Event Management:** Creating, editing, sharing, and deleting events, including importing match schedules from images using OCR.
- **Team Management:** Viewing and managing team details, scores, match history, statistics, autonomous paths, and alliance recommendations.
- **User Management:** Creating and managing user accounts, handling logins, managing permissions, and blocking users.
- **Data Storage:** Utilizing Firebase Realtime Database and Firestore for data persistence, along with local storage using shared preferences.
- **Push Notifications:** Delivering notifications for new event invitations. 

The app leverages various libraries and services, including Firebase, Google ML Kit, The Orange Alliance API, and ChatGPT, to provide a comprehensive and interactive user e

In [50]:
curr_string = '\n'.join([message['parts'][0] for interaction in team.context_threads['all'] for message in interaction.to_dict()])
(curr_tokens := pm.model.count_tokens(curr_string).total_tokens)

20746

In [51]:
would_be_string = '\n'.join([source_file_tree.nodes[node]['content'] for node in source_dependency_graph.nodes])
(would_be_tokens := pm.model.count_tokens(would_be_string).total_tokens)

122951

In [52]:
(percent_tokens_reduction := 1 - (curr_tokens / would_be_tokens))

0.8312661141430326

## Context Caching

In [53]:
pm.model.count_tokens('\n'.join(message['parts'][0] for interaction in team.context_threads['all'] for message in interaction.to_dict())).total_tokens

20746

In [54]:
# Cache the context threads
factor = 32768 / pm.model.count_tokens('\n'.join(message['parts'][0] for interaction in team.context_threads['all'] for message in interaction.to_dict())).total_tokens
# eg. if factor is 3.2, round to 4
factor = math.ceil(factor)
team.context_threads['all'] *= factor
pm.model.count_tokens('\n'.join(message['parts'][0] for interaction in team.context_threads['all'] for message in interaction.to_dict())).total_tokens

41469

In [55]:
pm.cache_context(team.context_threads['all'], ttl=datetime.timedelta(minutes=5))

In [58]:
response_format = f"""
You may choose ACTIONS from the following list:
- WRITE | $FILE_PATH$ | $PROMPT$ (PROMPT should inform how the new FILE_PATH should be written)
- EDIT | $FILE_PATH$ | $PROMPT$ (Edit the provided file to fulfill the prompt)
- DELETE | $FILE_PATH$ (Delete the provided file)
----------
Here is a sample response:
```
1. WRITE | {source.get_working_dir()}\\path\\to\\newfile.ext | Implement ... with the following properties:
    - property1
    - property2
2. EDIT | {source.get_working_dir()}\\path\\to\\file.ext | Fix the bug ... by doing:
    - step1
    - step2
3. DELETE | {source.get_working_dir()}\\path\\to\\some\\file.ext | Reasons for deletion:
    - reason1
    - reason2
```
"""

# Create new file tree
prompt = f'''
You are to build the following feature:
{feature}
------------------
{response_format}
------------------
You need to follow the response template structure exactly or the response will not be accepted.
'''
print(response := pm.chat(prompt))

```
1. WRITE | lib/models/Organization.dart | Create a new class named `Organization` with the following properties:
    - `id`: String (backend-generated unique ID)
    - `name`: String
    - `teamNumber`: int
    - `profilePicture`: String? (optional URL for profile picture)
    - `description`: String? (optional description)
    - `location`: String? (optional location)

2. WRITE | lib/providers/OrganizationProvider.dart | Create a new provider named `OrganizationProvider` that:
    - Uses Firebase Firestore to store and manage organizations.
    - Provides methods for:
        - Creating a new organization.
        - Fetching an organization by ID.
        - Updating organization details.
        - Listing all organizations.
        - Joining an organization.
        - Leaving an organization.
        - Listing members of an organization.

3. WRITE | lib/views/organization/OrganizationList.dart | Create a new widget named `OrganizationList` that:
    - Displays a list of organizati

In [62]:
import re

pattern  = re.compile(r'^\d+\.\s*((?:.*\n?)*?(?=\n\d+\.|\Z))', re.MULTILINE)
md = extract_markdown_blocks(response)[0]
lines = pattern.findall(md)
actions = [(action[0].strip(), action[1].strip()[len(source.get_working_dir()) + 1:], action[2].strip()) for line in lines if (action := line.split('|'))]
for action in actions:
    print(action)

('WRITE', 'models/Organization.dart', 'Create a new class named `Organization` with the following properties:\n    - `id`: String (backend-generated unique ID)\n    - `name`: String\n    - `teamNumber`: int\n    - `profilePicture`: String? (optional URL for profile picture)\n    - `description`: String? (optional description)\n    - `location`: String? (optional location)')
('WRITE', 'providers/OrganizationProvider.dart', 'Create a new provider named `OrganizationProvider` that:\n    - Uses Firebase Firestore to store and manage organizations.\n    - Provides methods for:\n        - Creating a new organization.\n        - Fetching an organization by ID.\n        - Updating organization details.\n        - Listing all organizations.\n        - Joining an organization.\n        - Leaving an organization.\n        - Listing members of an organization.')
('WRITE', 'views/organization/OrganizationList.dart', 'Create a new widget named `OrganizationList` that:\n    - Displays a list of organ

## Batch Processing

In [66]:
# We will group all consecutive actions (of identical type) together
grouped = []
current_group = [actions[0]]

for i in range(1, len(actions)):
    if actions[i][0] == current_group[-1][0]:
        current_group.append(actions[i])
    else:
        grouped.append(current_group)
        current_group = [actions[i]]

grouped.append(current_group)
for group in grouped:
    print(group)

[('WRITE', 'models/Organization.dart', 'Create a new class named `Organization` with the following properties:\n    - `id`: String (backend-generated unique ID)\n    - `name`: String\n    - `teamNumber`: int\n    - `profilePicture`: String? (optional URL for profile picture)\n    - `description`: String? (optional description)\n    - `location`: String? (optional location)'), ('WRITE', 'providers/OrganizationProvider.dart', 'Create a new provider named `OrganizationProvider` that:\n    - Uses Firebase Firestore to store and manage organizations.\n    - Provides methods for:\n        - Creating a new organization.\n        - Fetching an organization by ID.\n        - Updating organization details.\n        - Listing all organizations.\n        - Joining an organization.\n        - Leaving an organization.\n        - Listing members of an organization.'), ('WRITE', 'views/organization/OrganizationList.dart', 'Create a new widget named `OrganizationList` that:\n    - Displays a list of or

## Identify Independant Actions

In [38]:
for group in grouped:
    for action, file, prompt in group:
        if action == 'WRITE':
            source_dependency_graph.add_node(file, content=prompt)
        elif action == 'EDIT':
            source_dependency_graph.add_node(file, content=prompt)
        elif action == 'DELETE':
            source_dependency_graph.remove_node(file)